In [2]:
# import raadpy as rp
import numpy as np
import pandas as pd

In [11]:
LocationFile = '..\..\TGF_data_for_research\CubeSat_Data\Light-1-LLA\LIGHT-1_LLA_April.csv'

LocationData = pd.read_csv(LocationFile)

pd.options.display.max_rows = None

LocationData
